In [1]:
import pandas as pd
import os
import json
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D

In [2]:
data_path = os.path.join(os.getcwd(), "data")
seller_items_path = os.path.join(data_path, "input", "seller_items")

In [3]:
# Lista para armazenar os dados
data = []

# Percorrer todos os arquivos no diretório
for filename in os.listdir(seller_items_path):
    if filename.endswith("_items.json"):  # Garantir que são arquivos de sellers
        file_path = os.path.join(seller_items_path, filename)

        # Abrir e ler o arquivo linha por linha
        with open(file_path, "r", encoding="utf-8") as file:
            for line in file:
                try:
                    item_data = json.loads(line)  # Converter JSON para dicionário
                    data.append(item_data)
                except json.JSONDecodeError as e:
                    print(f"Erro ao ler {filename}: {e}")

# Criar DataFrame do pandas
df = pd.DataFrame(data)

# Exibir as primeiras linhas
display(df.head())

,seller_id,category_id,price,original_price,discount,available_qty,cataloged,free_shipping,condition,installments,buying_mode,city,state,has_gtin,num_attributes
0,1001057832,MLA14407,169999.0,NaN,0.0,1,1,1,new,6,buy_it_now,Olivos,Buenos Aires,1,6
1,1001057832,MLA14407,189999.0,NaN,0.0,1,1,1,new,6,buy_it_now,Olivos,Buenos Aires,1,8
2,1001057832,MLA430383,99900.0,NaN,0.0,1,1,1,new,6,buy_it_now,Olivos,Buenos Aires,1,9
3,1001057832,MLA407557,129999.0,NaN,0.0,1,1,1,new,3,buy_it_now,Olivos,Buenos Aires,1,7
4,1001057832,MLA91758,167999.0,NaN,0.0,1,1,1,new,3,buy_it_now,Olivos,Buenos Aires,1,9


In [ ]:
# Grouping by seller_id and aggregating metrics
df_grouped = df.groupby("seller_id").agg(
    {
        "category_id": [
            lambda x: list(set(x)),
            lambda x: len(set(x)),
        ],  # Unique categories & count
        "price": ["count", "mean", "min", "max", "sum"],  # Stats for price
        "original_price": ["mean", "min", "max"],  # Stats for original price
        "discount": ["mean", "max"],  # Avg and max discount
        "available_qty": ["sum", "mean", "min", "max"],  # Stock metrics
        "cataloged": "sum",  # Number of cataloged items
        "free_shipping": "sum",  # Number of items with free shipping
        "condition": lambda x: x.value_counts().to_dict(),  # Count of "new"/"used"
        "installments": "mean",  # Avg installments
        "buying_mode": lambda x: x.value_counts().to_dict(),  # Count of different buying modes
        "city": lambda x: list(set(x)),  # Unique cities
        "state": lambda x: list(set(x)),  # Unique states
        "has_gtin": "sum",  # Total items with GTIN
        "num_attributes": "mean",  # Avg number of attributes per item
    }
)
#df=None
# Rename multi-index columns
df_grouped.columns = [
    "unique_categories",
    "num_categories",
    "total_items",
    "price_mean",
    "price_min",
    "price_max",
    "price_sum",
    "original_price_mean",
    "original_price_min",
    "original_price_max",
    "discount_mean",
    "discount_max",
    "available_qty_sum",
    "available_qty_mean",
    "available_qty_min",
    "available_qty_max",
    "cataloged_sum",
    "free_shipping_sum",
    "condition",
    "installments_mean",
    "buying_mode",
    "unique_cities",
    "unique_states",
    "has_gtin_sum",
    "num_attributes_mean",
]

df_grouped = df_grouped.reset_index()

# Compute proportions
df_grouped["proportion_cataloged"] = (
    df_grouped["cataloged_sum"] / df_grouped["total_items"]
)
df_grouped["proportion_free_shipping"] = (
    df_grouped["free_shipping_sum"] / df_grouped["total_items"]
)
df_grouped["proportion_has_gtin"] = (
    df_grouped["has_gtin_sum"] / df_grouped["total_items"]
)

# Compute proportions for "new" condition
df_grouped["proportion_new"] = df_grouped["condition"].apply(
    lambda x: x.get("new", 0) / sum(x.values()) if x else 0
)

# Compute proportion for "buy_it_now"
df_grouped["proportion_buy_it_now"] = df_grouped["buying_mode"].apply(
    lambda x: x.get("buy_it_now", 0) / sum(x.values()) if x else 0
)

# Get most frequent city
df_grouped["most_frequent_city"] = df_grouped["unique_cities"].apply(
    lambda x: max(set(x), key=x.count) if x else None
)

# Get most frequent state
df_grouped["most_frequent_state"] = df_grouped["unique_states"].apply(
    lambda x: max(set(x), key=x.count) if x else None
)

display(df_grouped.head())

In [ ]:
df_fields = df_grouped[
    [
        #"seller_id",
        "num_categories",
        "total_items",
        "price_mean",
        #"price_min",
        #"price_max",
        "price_sum",
        "original_price_mean",
        "original_price_min",
        "original_price_max",
        "discount_mean",
        #"discount_max",
        "available_qty_sum",
        "available_qty_mean",
        #"available_qty_min",
        #"available_qty_max",
        "cataloged_sum",
        "free_shipping_sum",
        "installments_mean",
        "has_gtin_sum",
        "num_attributes_mean",
        "proportion_cataloged",
        "proportion_free_shipping",
        "proportion_has_gtin",
        "proportion_new",
        "proportion_buy_it_now",
        "most_frequent_city",
        "most_frequent_state",
    ]
]
display(df_fields.head())

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy import stats

# 🔹 Passo 1: Selecionar as features relevantes
df_fields = df_grouped[
    [
        "num_categories",
        "total_items",
        "price_mean",
        "price_sum",
        "original_price_mean",
        "original_price_min",
        "original_price_max",
        "discount_mean",
        "available_qty_sum",
        "available_qty_mean",
        "cataloged_sum",
        "free_shipping_sum",
        "installments_mean",
        "has_gtin_sum",
        "num_attributes_mean",
        "proportion_cataloged",
        "proportion_free_shipping",
        "proportion_has_gtin",
        "proportion_new",
        "proportion_buy_it_now",
        "most_frequent_city",
        "most_frequent_state",
    ]
].copy()  # Criar uma cópia para evitar avisos de mutação

# 🔹 Passo 2: Aplicar Frequency Encoding nas variáveis categóricas
categorical_cols = ["most_frequent_city", "most_frequent_state"]
for col in categorical_cols:
    freq_map = df_fields[col].value_counts(normalize=True).to_dict()
    df_fields[col] = df_fields[col].map(freq_map)

df_fields = df_fields.fillna(0)

# # 🔹 Passo 3: Remover Outliers usando Z-score
# z_scores = np.abs(stats.zscore(df_fields))
# df_filtered = df_fields[
#     (z_scores < 3).all(axis=1)
# ]  # Mantém apenas valores dentro de 3 desvios padrão

# 🔹 Passo 4: Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_fields)

# 🔹 Passo 5: Encontrar o número ideal de clusters (Método do Cotovelo)
inertia = []
K_range = range(1, 16)  # Testando de 1 a 10 clusters
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

# 🔹 Passo 6: Plotar o Método do Cotovelo
plt.figure(figsize=(8, 6))
plt.plot(K_range, inertia, marker="o", linestyle="-")
plt.title("Método do Cotovelo")
plt.xlabel("Número de Clusters (k)")
plt.ylabel("Inertia")
plt.show()

In [ ]:
# 🔹 Passo 7: Aplicar K-Means com k escolhido (exemplo: k=4)
kmeans = KMeans(n_clusters=5, random_state=42)
df_fields["cluster"] = kmeans.fit_predict(X_scaled)

# 🔹 Passo 8: Visualizar os primeiros vendedores e seus clusters
df_fields.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 🔹 Passo 1: Reduzir para 3 dimensões com PCA
pca = PCA(n_components=3)
X_pca_3d = pca.fit_transform(X_scaled)

# 🔹 Passo 2: Normalizar os componentes principais
scaler_pca = StandardScaler()
X_pca_3d_normalized = scaler_pca.fit_transform(X_pca_3d)

# 🔹 Passo 3: Criar gráfico 3D dos clusters normalizados
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection="3d")

scatter = ax.scatter(
    X_pca_3d_normalized[:, 0],
    X_pca_3d_normalized[:, 1],
    X_pca_3d_normalized[:, 2],
    c=df_fields["cluster"],
    cmap="viridis",
    s=50,
)

# 🔹 Rótulos e barra de cores
ax.set_title("Clusters de Vendedores (PCA 3D - Normalizado)")
ax.set_xlabel("Componente Principal 1 (Normalizado)")
ax.set_ylabel("Componente Principal 2 (Normalizado)")
ax.set_zlabel("Componente Principal 3 (Normalizado)")
fig.colorbar(scatter, ax=ax, label="Cluster")

plt.show()

In [ ]:
import numpy as np

# Find the index of the highest value in PCA Component 1 and 2
highest_value_index = np.argmax(X_pca_3d_normalized[:, 0] + X_pca_3d_normalized[:, 1])

# Remove the outlier from the PCA-transformed dataset
X_pca_3d_normalized = np.delete(X_pca_3d_normalized, highest_value_index, axis=0)

# Remove the corresponding row from df_fields
df_fields = df_fields.drop(df_fields.index[highest_value_index]).reset_index(drop=True)

# Replot without outlier
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection="3d")

scatter = ax.scatter(
    X_pca_3d_normalized[:, 0],
    X_pca_3d_normalized[:, 1],
    X_pca_3d_normalized[:, 2],
    c=df_fields["cluster"],
    cmap="viridis",
    s=50,
)

ax.set_title("Clusters de Vendedores (PCA 3D - Sem Outlier)")
ax.set_xlabel("Componente Principal 1 (Normalizado)")
ax.set_ylabel("Componente Principal 2 (Normalizado)")
ax.set_zlabel("Componente Principal 3 (Normalizado)")
fig.colorbar(scatter, ax=ax, label="Cluster")

plt.show()

In [ ]:
# Obter a matriz de cargas (loadings)
pca_components = pd.DataFrame(
    pca.components_,  # Coeficientes do PCA
    columns=df_fields.drop(columns='cluster').columns,  # Nome das variáveis originais
    index=[f"PC{i+1}" for i in range(pca.n_components_)],  # Nome dos componentes
)

# Exibir a matriz de cargas dos 3 primeiros PCs
pca_components.T.sort_values(by="PC1", ascending=False)

In [ ]:
df_cluster_summary = df_fields.groupby("cluster").mean()
df_cluster_summary

In [ ]:
df_cluster_summary.T.sort_values(
    by=0, ascending=False
)  # Ordena pela média do cluster 0

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Variáveis numéricas para gerar boxplots
numeric_cols = [
    "num_categories",
    "total_items",
    "price_mean",
    "price_sum",
    "original_price_mean",
    "original_price_min",
    "original_price_max",
    "discount_mean",
    "available_qty_sum",
    "available_qty_mean",
    "cataloged_sum",
    "free_shipping_sum",
    "installments_mean",
    "has_gtin_sum",
    "num_attributes_mean",
    "proportion_cataloged",
    "proportion_free_shipping",
    "proportion_has_gtin",
    "proportion_new",
    "proportion_buy_it_now",
]

# Gerar boxplots para cada variável numérica
for col in numeric_cols:
    plt.figure(figsize=(12, 6))
    sns.boxplot(x=df_fields["cluster"], y=df_fields[col])
    plt.title(f"Distribuição de {col} por Cluster")
    plt.show()